During Part 1, I suspected that Part 2 would not work with a brute force solution (i.e. keeping a literal representation of the polymer), so I instead kept track of how often pairs showed up. Luckily that turned out to be the right call for Part 2!

In [1]:
import collections
from typing import Dict


class Polymer:
    
    def __init__(self, template: str, rule_map: Dict[str, str]):
        self.pair_count: Dict[str, int] = collections.defaultdict(int)
            
        for i in range(len(template)-1):
            pair = template[i:i+2]
            self.pair_count[pair] += 1

        self.char_count: Dict[str, int] = collections.defaultdict(int)
        for char in template:
            self.char_count[char] += 1
        
        self.rule_map = rule_map
        self.step_num = 0
        
    def step(self):
        new_pair_count: Dict[str, int] = collections.defaultdict(int)
        
        for rule_pair, rule_insert in self.rule_map.items():
            count = self.pair_count.get(rule_pair, 0)
            if count > 0:
                new_pair_count[rule_pair[0] + rule_insert] += count
                new_pair_count[rule_insert + rule_pair[1]] += count
                self.char_count[rule_insert] += count
        
        self.pair_count = new_pair_count
        self.step_num += 1
        
    def answer(self):
        sorted_elements = sorted(self.char_count.items(), key=lambda kv: kv[1])
        most_common = sorted_elements[-1]
        least_common = sorted_elements[0]
        
        return most_common[1] - least_common[1]

In [2]:
def create_polymer(input_filename: str) -> Polymer:
    with open(input_filename) as input_file:
        template, raw_rules = input_file.read().split("\n\n")

    rule_map = {}
    for rule in raw_rules.splitlines():
        pair, element = rule.split(" -> ")
        rule_map[pair] = element
    
    return Polymer(template, rule_map)

# Test Counting Logic

In [3]:
def test_counting_logic():
    polymer = create_polymer("input-example.txt")

    expected_polymers = {
        1: 'NCNBCHB',
        2: 'NBCCNBBBCBHCB',
        3: 'NBBBCNCCNBBNBNBBCHBHHBCHB',
        4: 'NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB',
    }

    for i in range(4):
        polymer.step()

        expected = expected_polymers[polymer.step_num]
        
        # Get expected values
        pair_count = collections.defaultdict(int)
        for i in range(len(expected)-1):
            pair = expected[i:i+2]
            pair_count[pair] += 1
        char_count = collections.Counter(expected)

        # Compare against values stored in `polymer`.
        assert polymer.pair_count == pair_count
        assert polymer.char_count == char_count
        
    for i in range(4, 10):
        polymer.step()
    assert polymer.answer() == 1588
    
    for i in range(10, 40):
        polymer.step()
    assert polymer.answer() == 2188189693529
        
    print("Test cases succeeded!")

test_counting_logic()

Test cases succeeded!


# Parts 1 and 2

In [4]:
polymer = create_polymer("input.txt")

for i in range(10):
    polymer.step()
print("Part 1 answer:", polymer.answer())

for i in range(30):
    polymer.step()
print("Part 2 answer:", polymer.answer())

Part 1 answer: 2233
Part 2 answer: 2884513602164
